In [1]:
from freelancersdk.session import Session
from freelancersdk.resources.users.users import search_freelancers, get_users, get_user_by_id, get_portfolios
from freelancersdk.resources.users.exceptions import \
    UsersNotFoundException, PortfoliosNotFoundException
from freelancersdk.resources.users.helpers import (
    create_get_users_details_object, create_get_users_object
)
from freelancersdk.resources.projects.projects import get_jobs
from freelancersdk.resources.projects.exceptions import \
    JobsNotFoundException
import os
import pandas as pd
import json

# Freelancer User Data

- The api is limited to returning 100 rows only for user data
- Filter user_df by 'closed' == False to exclude any closed accounts
- Exclude columns in this  list

  exclude_cols = [
    'limited_account',
    'pool_ids',
    'enterprise_ids',
    'escrowcom_account_linked',
    'escrowcom_interaction_required',
    'enterprises',
    'oauth_password_credentials_allowed',
    'registration_completed',
    'is_profile_visible',
    'registration_date',
    'chosen_role',
    'role',
    'status',
    'timezone'
  ]

In [2]:
oauth_token='PaWtwT7yMjkaPXEbODfOfQLHuSSEYE'
url='https://www.freelancer.com'
search_keywords = ['engineer', 'data', 'design', 'web']

exclude_user_cols = [
    'limited_account',
    'pool_ids',
    'enterprise_ids',
    'escrowcom_account_linked',
    'escrowcom_interaction_required',
    'enterprises',
    'oauth_password_credentials_allowed',
    'registration_completed',
    'is_profile_visible',
    'registration_date',
    'chosen_role',
    'role',
    'status',
    'timezone'
]

In [3]:
def get_fln_session():
    # url = os.environ.get('FLN_URL')
    # oauth_token = os.environ.get('FLN_OAUTH_TOKEN')
    return Session(oauth_token=oauth_token, url=url)

session = get_fln_session()

In [4]:
def search_freelance_users(session, query):
    user_details = create_get_users_details_object(
        country=True,
        status=True,
        display_info=True,
        profile_description=True,
        qualifications=True,
        portfolio=True,
        jobs=True,
        reputation=True,
        corporate_accounts=True
    )
    try:
        result = search_freelancers(
            session=session,
            query=query,
            user_details=user_details,
            limit=100 # maximum results = 100
        )
    except UsersNotFoundException as e:
        print('Error message: {}'.format(e.message))
        print('Server response: {}'.format(e.error_code))
        return None
    else:
        return result

In [28]:
def create_user_df(session, search_keywords, exclude_cols):
    df_list = [pd.DataFrame.from_dict(search_freelance_users(session,keyword)['users']) for keyword in search_keywords]
    # Concatenate list into one consolidated dataframe
    df = pd.concat(df_list, ignore_index=True)
    print(df.shape)
    # Drop all excluded columns
    df.drop(exclude_cols, axis=1, inplace=True)
    print(df.shape)
    print(type(df))
    df.drop_duplicates(inplace=True, ignore_index=True)
    print(f"User dataframe shape: {df.shape}")
    return df
    # # Export csv
    # final_user_df.to_csv('data/raw/fln_users.csv', index=False)
    # # !!!!!!!TO DO deduplicate rows in the final_user_df!!!!!!!
    # # !!!!!!!Get a unique list of user ids to call other apis!!!!!!!

In [29]:
user_df = create_user_df(session, ['data', 'test'], exclude_user_cols)
user_df.head()

(200, 31)
(200, 17)
<class 'pandas.core.frame.DataFrame'>


TypeError: unhashable type: 'dict'

In [5]:
# Get a list of dataframes by searching keywords one by one
df_list = [pd.DataFrame.from_dict(search_freelance_users(session,keyword)['users']) for keyword in search_keywords]

In [6]:
# Concatenate list into one consolidated dataframe
user_df = pd.concat(df_list, ignore_index=True)

In [7]:
# Drop all excluded columns
final_user_df = user_df.drop(exclude_user_cols, axis=1)
final_user_df

,id,username,closed,reputation,jobs,profile_description,hourly_rate,display_name,tagline,location,primary_currency,qualifications,primary_language,portfolio_count,public_name,company,corporate
0,31295818,furkancakir,False,{'entire_history': {'overall': 4.9828062052387...,"[{'id': 6, 'name': 'C Programming', 'category'...",Hi. I'm Senior Mechanical Design Engineer and ...,70.0,furkancakir,Senior Mechanical Design Engineer,"{'country': {'name': 'Turkey', 'flag_url': '/i...","{'id': 1, 'code': 'USD', 'sign': '$', 'name': ...","[{'id': 492, 'name': 'Turkish - Level 1', 'lev...",en,10,Furkan Taha C.,,NaN
1,33481141,VisualStudio77,False,{'entire_history': {'overall': 4.7516860802426...,"[{'id': 42, 'name': 'Engineering', 'category':...",******A complete Building ...,50.0,American HomeEngineering,"US Licensed Architect, Structural Engineer- Pe...","{'country': {'name': 'Bangladesh', 'flag_url':...","{'id': 1, 'code': 'USD', 'sign': '$', 'name': ...","[{'id': 533, 'name': 'Preferred Freelancer Pro...",en,9,American HomeEngineering,Archi Engineering Studio,"{'status': 'active', 'founder_id': 62421482}"
2,39824241,JaneBurke,False,{'entire_history': {'overall': 4.9887323659978...,"[{'id': 42, 'name': 'Engineering', 'category':...",I'm Neamat manager of a specialized Architect ...,50.0,JaneBurke,Licensed Architect/PE/ME team at USA+Canada+Aus.,"{'country': {'name': 'Bangladesh', 'flag_url':...","{'id': 1, 'code': 'USD', 'sign': '$', 'name': ...","[{'id': 533, 'name': 'Preferred Freelancer Pro...",en,16,Md Neamat Ullah H.,,NaN
3,27245644,DavidJG23,False,{'entire_history': {'overall': 4.9999999999999...,"[{'id': 44, 'name': 'Mobile App Development', ...",Welcome to my profile!\n\nI’m David Jimenez Gu...,50.0,DavidJG23,iOS Engineer,"{'country': {'name': 'Andorra', 'flag_url': '/...","{'id': 8, 'code': 'EUR', 'sign': '€', 'name': ...","[{'id': 160, 'name': 'US English - Level 3', '...",en,0,David J.,,NaN
4,34205968,dervish11,False,{'entire_history': {'overall': 4.8902325275643...,"[{'id': 20, 'name': 'Graphic Design', 'categor...",Architectural & Civil Engineer | Fusing Creati...,50.0,dervish11,Architectural & Engineering,"{'country': {'name': 'Albania', 'flag_url': '/...","{'id': 8, 'code': 'EUR', 'sign': '€', 'name': ...","[{'id': 69, 'name': 'Freelancer Orientation', ...",en,39,Dervish M.,,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,460831,WhizPool,False,{'entire_history': {'overall': 4.9986401676773...,"[{'id': 3, 'name': 'PHP', 'category': {'id': 1...","Hello,\n\nThank you for considering WhizPool f...",30.0,WhizPool,"Mobile Apps, Big data, Cloud and Web consultants.","{'country': {'name': 'Pakistan', 'flag_url': '...","{'id': 1, 'code': 'USD', 'sign': '$', 'name': ...","[{'id': 3, 'name': 'General Orientation', 'lev...",en,26,WhizPool,Whizpool,"{'status': 'expired', 'founder_id': 32255102}"
396,20677081,TheEminence,False,{'entire_history': {'overall': 4.9910434306037...,"[{'id': 3, 'name': 'PHP', 'category': {'id': 1...","Welcome to TETS, is a Software Testing company...",9.0,TETS,QA Automation ✅Manual✅Performance✅Security✅Devops,"{'country': {'name': 'India', 'flag_url': '/im...","{'id': 11, 'code': 'INR', 'sign': '₹', 'name':...","[{'id': 533, 'name': 'Preferred Freelancer Pro...",en,15,TETS,The Eminence technology solutions,"{'status': 'active', 'founder_id': 24696992}"
397,17839766,SeoQueen786,False,{'entire_history': {'overall': 4.9666668181172...,"[{'id': 3, 'name': 'PHP', 'category': {'id': 1...",**Maximize Your Online Presence!**\n\nDigital...,12.0,SeoQueen786,Digital Marketing | Wordpress | Shopify | PHP,"{'country': {'name': 'India', 'flag_url': '/im...","{'id': 11, 'code': 'INR', 'sign': '₹', 'name':...","[{'id': 533, 'name': 'Preferred Freelancer Pro...",en,37,SeoQueen786,,"{'status': 'active', 'founder_id': 25797756}"
398,51841746,SEOIdeal,False,{'entire_history': {'overall': 4.9697077105673...,"[{'id': 17, 'name': 'Website Design', 'categor...","I am Minakshi Mewara, Digital Marketing Ma

In [30]:
final_user_df.drop_duplicates()

TypeError: unhashable type: 'dict'

In [54]:
print(final_user_df.shape)
# Export csv
final_user_df.to_csv('data/raw/fln_users.csv', index=False)

(400, 17)


## Freelancers final_user_df data exploration

In [9]:
final_user_df['jobs'][123]

[{'id': 9,
  'name': 'JavaScript',
  'category': {'id': 1, 'name': 'Websites, IT & Software'},
  'seo_url': 'javascript',
  'local': False},
 {'id': 10,
  'name': 'XML',
  'category': {'id': 1, 'name': 'Websites, IT & Software'},
  'seo_url': 'xml',
  'local': False},
 {'id': 14,
  'name': 'Visual Basic',
  'category': {'id': 1, 'name': 'Websites, IT & Software'},
  'seo_url': 'visual_basic',
  'local': False},
 {'id': 15,
  'name': '.NET',
  'category': {'id': 1, 'name': 'Websites, IT & Software'},
  'seo_url': 'dot_net',
  'local': False},
 {'id': 29,
  'name': 'Windows Desktop',
  'category': {'id': 1, 'name': 'Websites, IT & Software'},
  'seo_url': 'windows',
  'local': False},
 {'id': 38,
  'name': 'SEO',
  'category': {'id': 1, 'name': 'Websites, IT & Software'},
  'seo_url': 'seo',
  'local': False},
 {'id': 55,
  'name': 'Excel',
  'category': {'id': 4, 'name': 'Data Entry & Admin'},
  'seo_url': 'excel',
  'local': False},
 {'id': 67,
  'name': 'Testing / QA',
  'category': {

In [14]:
df.to_string(max_colwidth=None)

'         id          username  closed                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

In [8]:
user_ids = final_user_df['id'].unique().tolist()
len(user_ids)

285

# Get Portfolio

In [11]:
def sample_get_portfolios(session, user_ids):
    user_ids = user_ids
    try:
        portfolios = get_portfolios(session, user_ids=user_ids)
    except PortfoliosNotFoundException as e:
        print('Error message: {}'.format(e.message))
        print('Server: response: {}'.format(e.error_code))
        return None
    else:
        return portfolios

result = sample_get_portfolios(session, user_ids)


In [22]:
exclude_portfolio_cols = [
    'content_type',
    'featured',
    'files',
    'articles',
    'jobs',
    'categories',
    'last_modify_date',
    'position'
]

# Transform the data such that it can be converted into a data
def create_portfolio_df(result, exclude_cols):
    # Rearrange the api results dict such that each row is a portfolio id with user id as a foreign key
    if result:
        portfolios = []
        users = result['users'].keys()
        for k in users:
            portfolios+=(result['portfolios'][k])
    print('Got {} portfolios from {} users.'.format(len(portfolios), len(users)))

    df = pd.DataFrame.from_records(portfolios)
    return df.drop(exclude_portfolio_cols, axis=1)

In [23]:
portfolio_df = create_portfolio_df(result, exclude_portfolio_cols)
portfolio_df.head()

Got 911 portfolios from 100 users.


,id,user_id,title,description
0,4244666,4193,Golden Routes | Travel App,"According to the requirements of the clients, ..."
1,5988328,4193,Classified Marketplace Website,The development of the classified website was ...
2,6034019,4193,Ecommerce (Clothing),"When building an eCommerce website, we look in..."
3,4244726,4193,Event Listing and booking platform,"According to the client's requirement, the sys..."
4,1706811,4193,Caravan listing web site,We use Html5+Css3 to ensure the website fits o...


In [25]:
result = portfolio_df.query('id==6034019')

print(result.to_string(max_colwidth=None))

        id  user_id                 title                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              description
2  6034019     4193  Ecommerce (Clothing)  When building an eCommerce website, we look into every detail that matters for your bottom line. From page load speed to mobile-friendliness to navigation structure, each facet of development and design will impact user experience and the number of sales generated in a given month – by a wide margin. Even details

# Jobs API

This is not used 

In [70]:
def sample_get_jobs(session,job_ids):
    get_jobs_data = {
        'job_ids': job_ids,
        'seo_details': True,
        'lang': 'en',
    }
    try:
        j = get_jobs(session, **get_jobs_data)
    except JobsNotFoundException as e:
        print('Error message: {}'.format(e.message))
        print('Server response: {}'.format(e.error_code))
        return None
    else:
        return j


j = sample_get_jobs([186, 188, 193, 271])
if j:
    print('Found jobs: {}'.format(len(j)))

Found jobs: 4


In [72]:
df4 = pd.DataFrame.from_records(j)
df4

,id,name,category,active_project_count,seo_url,seo_info,local
0,186,Civil Engineering,"{'id': 5, 'name': 'Engineering & Science'}",None,civil_engineering,{'seo_text': '<p>Civil engineering a broad and...,False
1,188,Structural Engineering,"{'id': 5, 'name': 'Engineering & Science'}",None,structural_engineering,{'seo_text': '<p>Why hire a structural enginee...,False
2,193,Finite Element Analysis,"{'id': 5, 'name': 'Engineering & Science'}",None,finite_element_analysis,{'seo_text': '<p>Finite Element Analysis (FEA)...,False
3,271,AutoCAD,"{'id': 5, 'name': 'Engineering & Science'}",None,autocad,{'seo_text': '<p>AutoCAD is one of the most ub...,False


In [74]:
df4['seo_info'][0]

{'seo_text': "<p>Civil engineering a broad and expansive professional engineering discipline which is focused on designing, creating and maintaining the infrastructures all around us that bridge the connection between the human experience and the natural world just outside our perspective. Some typical civil engineering projects could be the construction of roadways, canals, bridges, culverts, dams. Civil engineering has a longstanding history in both the private and public sectors. It is practised by individuals, organizations, firms, projects may be sponsored by local, state, national or international governments, the military; the sky is really the limit when it comes to civil engineering projects. A very multidisciplinary focus in most cases, civil engineers are highly skilled at navigating the complexity and nuance of the technical requirements of their projects while also knowing how to work with the human systems and interfaces that are largely at the center of funding and admin